# AXIN1 Craniometadiaphyseal osteosclerosis with hip dysplasia  individuals
Date derived from [Terhal P, et al. AXIN1 bi-allelic variants disrupting the C-terminal DIX domain cause craniometadiaphyseal osteosclerosis with hip dysplasia. Am J Hum Genet. 2023](https://pubmed.ncbi.nlm.nih.gov/37582359/)

In [1]:
import pandas as pd
from IPython.display import display, HTML
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
from pyphetools.creation import *
from pyphetools.visualization import *
from pyphetools.validation import *
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.9.54


In [2]:
parser = HpoParser(hpo_json_file="../hp.json")
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
created_by="ORCID:0000-0002-0736-9199"
print(f"HPO version {hpo_version}")

HPO version 2024-02-27


In [3]:
df = pd.read_excel("input/AXIN1_individuals.xlsx")
df.head(2)

,PMID,title,individual_id,comment,disease_id,disease_label,transcript,allele_1,allele_2,variant.comment,...,Hypoplasia of the corpus callosum,Ectopic anterior pituitary gland,Arachnoid cyst,Brain atrophy,Macrocephaly,Relative macrocephaly,Selective mutism,Increased bone mineral density,Choanal stenosis,Miscellaneous
0,str,str,str,optional str,str,str,str,str,str,optional str,...,HP:0002079,HP:0012731,HP:0100702,HP:0012444,HP:0000256,HP:0004482,HP:5200001,HP:0011001,HP:0000452,Miscellaneous
1,PMID:37582359,AXIN1 bi-allelic variants disrupting the C-terminal DIX domain cause craniometadiaphyseal osteosclerosis with hip dysplasia,F1-II-1,NaN,OMIM:620558,Craniometadiaphyseal osteosclerosis with hip dysplasia,NM_003502.4,c.2395dup,c.2395dup,NM_181050.3:c.2395dupG,...,excluded,excluded,excluded,excluded,observed,na,observed,observed,observed,NaN


In [4]:
encoder = CaseTemplateEncoder(df=df, hpo_cr=hpo_cr, created_by=created_by)
individuals = encoder.get_individuals()

Created encoders for 39 fields


In [5]:
AXIN1_transcript = "NM_003502.4"
vmanager = VariantManager(df=df,
                          individual_column_name="individual_id",
                          gene_symbol="AXIN1",
                          transcript=AXIN1_transcript,
                          allele_1_column_name="allele_1",
                          allele_2_column_name="allele_2")

In [6]:
vmanager.to_summary()

,status,count,alleles
0,mapped,3,"c.2521C>T, c.2167C>T, c.2395dup"
1,unmapped,0,


In [7]:
vmanager.add_variants_to_individuals(individuals)

In [8]:
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1,
                                allelic_requirement=AllelicRequirement.BI_ALLELIC)
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

Level,Error category,Count
ERROR,CONFLICT,2


In [9]:
individuals = cvalidator.get_error_free_individual_list()
table = IndividualTable(individuals)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
F1-II-1 (FEMALE; P15Y6M),Craniometadiaphyseal osteosclerosis with hip dysplasia (OMIM:620558),NM_003502.4:c.2395dup (homozygous),Hypertelorism (HP:0000316); Depressed nasal bridge (HP:0005280); Hip dysplasia (HP:0001385); Patent ductus arteriosus (HP:0001643); Thickened calvaria (HP:0002684); Sandwich appearance of vertebral bodies (HP:0004618); Thickened cortex of long bones (HP:0000935); Flared metaphysis (HP:0003015); Coarse metaphyseal trabecularization (HP:0100670); Pachygyria (HP:0001302); Macrocephaly (HP:0000256); Selective mutism (HP:5200001); Increased bone mineral density (HP:0011001); Choanal stenosis (HP:0000452); excluded: Global developmental delay (HP:0001263); excluded: Atrial septal defect (HP:0001631); excluded: Ventricular septal defect (HP:0001629); excluded: Cephalohematoma (HP:0012541); excluded: Hypoplasia of the corpus callosum (HP:0002079); excluded: Ectopic anterior pituitary gland (HP:0012731); excluded: Arachnoid cyst (HP:0100702); excluded: Brain atrophy (HP:0012444)
F1-II-4 (FEMALE; P8Y3M),Craniometadiaphyseal osteosclerosis with hip dysplasia (OMIM:620558),NM_003502.4:c.2395dup (homozygous),Hypertelorism (HP:0000316); Depressed nasal bridge (HP:0005280); Hip dysplasia (HP:0001385); Thickened calvaria (HP:0002684); Sandwich appearance of vertebral bodies (HP:0004618); Thickened cortex of long bones (HP:0000935); Flared metaphysis (HP:0003015); Coarse metaphyseal trabecularization (HP:0100670); Cephalohematoma (HP:0012541); Macrocephaly (HP:0000256); Selective mutism (HP:5200001); Increased bone mineral density (HP:0011001); excluded: Global developmental delay (HP:0001263); excluded: Patent ductus arteriosus (HP:0001643); excluded: Atrial septal defect (HP:0001631); excluded: Ventricular septal defect (HP:0001629); excluded: Pachygyria (HP:0001302); excluded: Hypoplasia of the corpus callosum (HP:0002079); excluded: Ectopic anterior pituitary gland (HP:0012731); excluded: Arachnoid cyst (HP:0100702); excluded: Brain atrophy (HP:0012444); excluded: Choanal stenosis (HP:0000452)
F2-III-2 (MALE; P7Y1M),Craniometadiaphyseal osteosclerosis with hip dysplasia (OMIM:620558),NM_003502.4:c.2167C>T (homozygous),Global developmental delay (HP:0001263); Hypertelorism (HP:0000316); Depressed nasal bridge (HP:0005280); Atrial septal defect (HP:0001631); Ventricular septal defect (HP:0001629); Sandwich appearance of vertebral bodies (HP:0004618); Coarse metaphyseal trabecularization (HP:0100670); Hypoplasia of the corpus callosum (HP:0002079); Ectopic anterior pituitary gland (HP:0012731); Macrocephaly (HP:0000256); Increased bone mineral density (HP:0011001); excluded: Hip dysplasia (HP:0001385); excluded: Patent ductus arteriosus (HP:0001643); excluded: Pachygyria (HP:0001302); excluded: Cephalohematoma (HP:0012541); excluded: Arachnoid cyst (HP:0100702); excluded: Brain atrophy (HP:0012444); excluded: Selective mutism (HP:5200001); excluded: Choanal stenosis (HP:0000452)
F3-III-1 (FEMALE; P15Y6M),Craniometadiaphyseal osteosclerosis with hip dysplasia (OMIM:620558),NM_003502.4:c.2521C>T (homozygous),Hypertelorism (HP:0000316); Depressed nasal bridge (HP:0005280); Hip dysplasia (HP:0001385); Thickened calvaria (HP:0002684); Sandwich appearance of vertebral bodies (HP:0004618); Thickened cortex of long bones (HP:0000935); Coarse metaphyseal trabecularization (HP:0100670); Arachnoid cyst (HP:0100702); Macrocephaly (HP:0000256); Increased bone mineral density (HP:0011001); excluded: Global developmental delay (HP:0001263); excluded: Motor delay (HP:0001270); excluded: Patent ductus arteriosus (HP:0001643); excluded: Atrial septal defect (HP:0001631); excluded: Ventricular septal defect (HP:0001629); excluded: Pachygyria (HP:0001302); excluded: Cephalohematoma (HP:0012541); excluded: Hypoplasia of the corpus callosum (HP:0002079); excluded: Ectopic anterior pituitary gland (HP:0012731); excluded: Brain atrophy (HP:0012444); excluded: Selective mutism (HP:5200001); ex

In [10]:
encoder.output_individuals_as_phenopackets(individual_list=individuals)

We output 7 GA4GH phenopackets to the directory phenopackets


In [11]:
ingestor = PhenopacketIngestor(indir="phenopackets")
ppkt_d = ingestor.get_phenopacket_dictionary()
ppkt_list = list(ppkt_d.values())


[pyphetools] Ingested 7 GA4GH phenopackets.


In [12]:
builder = HpoaTableBuilder(phenopacket_list=ppkt_list)

In [13]:
PMID = "PMID:37582359" # Terhal et al., 2023
builder.autosomal_recessive(PMID)
hpoa_table_creator = builder.build()
df = hpoa_table_creator.get_dataframe()

We found a total of 28 unique HPO terms
Extracted disease: Craniometadiaphyseal osteosclerosis with hip dysplasia (OMIM:620558)


In [14]:
hpoa_table_creator.write_data_frame()

Wrote HPOA disease file to OMIM-620558.tab
